# SQL Chain, based on `SQLDatabaseChain`

## Setup

### Imports

In [1]:
import os

import db_connect 

from langsmith import Client

from langchain_community.llms.ollama import Ollama

from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain_community.llms.gigachat import GigaChat

### LangSmith

In [ ]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Models

In [2]:
gigachat = GigaChat(credentials=os.environ["GIGACHAT_AUTH"], verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

llama3 = Ollama(model="llama3:8b", temperature=0)
llama3_inst = Ollama(model="llama3:instruct", temperature=0)
llama3_inst_q8 = Ollama(model="llama3:8b-instruct-q8_0", temperature=0)
llama3_inst_fp16 = Ollama(model="llama3:8b-instruct-fp16", temperature=0)

In [3]:
gigachat.invoke("write a joke")

'Q: Why did the chicken cross the road?\nA: To get to the other side, of course!'

### Connect to DB

In [4]:
db = db_connect.get_db()

#### Check connection

In [5]:
db.run("SELECT * FROM passenger")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

In [6]:
db_chain = SQLDatabaseChain.from_llm(gigachat, db, verbose=True, use_query_checker=True)

In [7]:
res = db_chain.invoke("Select the names of all the people who are in the airline database?")



> Entering new SQLDatabaseChain chain...
Select the names of all the people who are in the airline database?
SQLQuery:SELECT passenger_name FROM passenger;
SQLResult: [('John',), ('James',), ('Poul',), ('Christofer',), ('Superman',)]
Answer:John, James, Poul, Christofer, Superman
> Finished chain.


In [8]:
print(res)

{'query': 'Select the names of all the people who are in the airline database?', 'result': 'John, James, Poul, Christofer, Superman'}
